In [ ]:
import sys
sys.path.insert(1, '../Codes_1Dtree')
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" 
CUDA_LAUNCH_BLOCKING=1

Load libraries

In [2]:
import torch
from torch_geometric.loader import DataLoader
from data.graph_dataset import *
from networks.gcnv6 import RecurrentFormulationNet, GraphNet
import matplotlib.pyplot as plt
from torch_geometric.loader import NeighborLoader
from networks.losses import LpLoss

In [3]:
class objectview(object):
    def __init__(self, d) -> None:
        self.__dict__ = d
    def setattr(self, attr_name, attr_value):
        self.__dict__[attr_name] = attr_value

args = objectview({
    'n_fields': 1,
    'n_meshfields': (13, 0),
    'hidden_size': 512,
    'n_layers': 10,
    'n_timesteps': 201,
    'n_previous_timesteps': 1,
    'aggr': 'sum',
    'act': 'relu',
    'dropout': 0.2,
    # 'use_hidden': True,
    'device': torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'),
    'lr': 5e-5,
    # 'lr_decay': 0.5,
    # 'lr_decay_period': 50,
    'weight_decay': 5e-4,
    'n_epoch': 50000,
    'alpha': 1.0,
    'batch_size': 100,
    'timestep': 201,
    'timeslice_hops': 0,
    'timeslice_steps': 1,
    'n_data_per_batch': 1,
    'forward_sequence': False,
    'criterion': LpLoss(),
    'plot': False
})

Load dataset

In [4]:
# dataset = OneDDatasetBuilder(
#     raw_dir='/data1/tam/datasets',
#     # root_dir='/data1/tam/downloaded_datasets_node_features',
#     root_dir='/data1/tam/test_datasets',
#     sub_dir='processed',
#     subjects=['10081'],
#     time_names=[str(i).zfill(3) for i in range(201)],
#     data_type = torch.float64,
#     readme='edge_index(2xn_edge), node_attr(n_nodex10), pressure(n_nodex201)'
# )
dataset = OneDDatasetLoader(
    root_dir='/data1/tam/downloaded_datasets_node_features',
    # root_dir='/data1/tam/downloaded_datasets_edge_node_separated',
    sub_dir='processed',
    subjects='all',
    time_names=[str(i).zfill(3) for i in range(201)],
    data_type = torch.float32
)

In [1]:
# # Try clipping output for better distribution
# dataset = normalize(
#     dataset=dataset,
#     sub_dir='normalized',
#     scaler_dict={
#         'node_attr': ('minmax_scaler', 0, None),
#         # 'edge_attr': ('minmax_scaler', 0, None),
#         'pressure': ('minmax_scaler', None, None),
#         # 'boundary_condition': ('minmax_scaler', None, None),
#         'flowrate': ('robust_scaler', None, None),
#         # 'pressure_dot': ('minmax_scaler', None, None),
#         # 'flowrate_dot': ('robust_scaler', None, None),
#         # 'time': ('minmax_scaler', None, None)
#     }
# )
# dataset = OneDDatasetLoader(
#     root_dir='/data1/tam/downloaded_datasets_node_features',
#     sub_dir='normalized',
#     subjects='all',
#     time_names=[str(i).zfill(3) for i in range(201)],
#     data_type = torch.float32
# )

In [5]:
dataset = batchgraph_generation_wise(
    dataset,
    sub_dir='batched',
    batch_gens=[[0, 5, 1]],
    timestep=args.timestep,
    timeslice_hops=args.timeslice_hops,
    timeslice_steps=args.timeslice_steps
)
# dataset = batchgraph_neighbor(
#     dataset,
#     sub_dir='batched',
#     batch_size=args.batch_size
# )
dataset = OneDDatasetLoader(
    root_dir='/data1/tam/downloaded_datasets_node_features',
    sub_dir='batched',
    subjects='all',
    time_names=[str(i).zfill(3) for i in range(201)],
    data_type = torch.float32
)

IndexError: list index out of range

In [ ]:
dataset

In [ ]:
dataset[0]

In [ ]:
(train_loader) = dataset_to_loader(
    dataset=dataset,
    data_subset_dict={
        'train': list(range(0, 2))
    },
    n_data_per_batch=100
)

In [ ]:
# fold_size = 2
# fold = [list(range(i*fold_size, (i+1)*fold_size)) for i in range(8)]
# n_train_fold = 5

# for i in range(n_train_fold):
#     test_subset = fold[i]
#     train_subset = []
#     for j in range(n_train_fold):
#         if j != i:
#             train_subset += fold[j]

(train_loader, test_loader) = dataset_to_loader(
    dataset=dataset,
    data_subset_dict={
        'train': list(range(0, 30)),
        'test': list(range(30, 35))
    },
    n_data_per_batch=args.n_data_per_batch
)

# train_set, test_set = dataset_to_loader(
#     dataset=dataset,
#     data_subset_dict={
#         'train': list(range(6, 36)),
#         'test': list(range(0, 5))
#     },
#     n_data_per_batch=args.n_data_per_batch
# )

Model initializing and training

In [ ]:
# model = GraphNet(
#     n_fields=args.n_fields,
#     n_meshfields=args.n_meshfields,
#     hidden_size=args.hidden_size,
#     n_layers=args.n_layers,
#     n_previous_timesteps=args.n_previous_timesteps,
#     # act=args.act,
#     dropout=args.dropout,
#     # use_hidden=args.use_hidden
# )
# setattr(model, 'name', 'model_GCN_LSTM')
# model = model.to(args.device)
# model.load_state_dict(torch.load(f'models/{model.name}_node2_epoch140.pth', map_location=args.device) )
# optimizer1 = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
# optimizer2 = torch.optim.SGD(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
# setattr(args, 'optimizer', optimizer2)

In [ ]:
model = GraphNet(
    n_fields=args.n_fields,
    n_meshfields=args.n_meshfields,
    n_timesteps=args.n_timesteps,
    hidden_size=args.hidden_size,
    n_layers=args.n_layers,
    n_previous_timesteps=args.n_previous_timesteps,
    # act=args.act,
    dropout=args.dropout,
    # use_hidden=args.use_hidden
)
setattr(model, 'name', 'model_GraphUNet')
model = model.to(args.device)
# model.load_state_dict(torch.load(f'models/{model.name}_node2_epoch6000.pth', map_location=args.device) )
optimizer1 = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
setattr(args, 'optimizer', optimizer1)

In [ ]:
def train(model, data, args):
    # optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
    ##
    F_true = torch.cat([data.pressure.unsqueeze(2)], dim=2) \
                .float().to(args.device)
    ##
    edge_index = torch.cat([data.edge_index, torch.flip(data.edge_index, dims=[0])], dim=1).to(args.device)
    
    ##
    meshfield = (data.node_attr.float().to(args.device), None)

    ##
    model.train()
    args.optimizer.zero_grad()

    F_pred = model.forward(
        edge_index=edge_index,
        meshfield=meshfield
    )
    ##
    
    loss = args.criterion(F_pred, F_true)
    loss.backward()
    args.optimizer.step()
    
    return loss.item()

def eval(model, data, args):
    ##
    F_true = torch.cat([data.pressure.unsqueeze(2)], dim=2) \
                .float().to(args.device)

    ##
    edge_index = torch.cat([data.edge_index, torch.flip(data.edge_index, dims=[0])], dim=1).to(args.device)
    ##
    meshfield = (data.node_attr.float().to(args.device), None)
    ##
    model.eval()

    with torch.no_grad():
        
        F_pred = model.forward(
            edge_index=edge_index,
            meshfield=meshfield
        )
    ##
    loss = args.criterion(F_pred, F_true)
        
    return loss.item()

In [ ]:
# Training
start_epoch = 0
for epoch in range(args.n_epoch):
    CUDA_LAUNCH_BLOCKING=1
    torch.cuda.empty_cache()

    ##
    # if epoch % 100 == 0:
    #     if args.n_timesteps < 100:
    #         args.n_timesteps *= 2
    train_loss = 0
    for i in range(train_loader.__len__()):
        data = next(iter(train_loader))
        train_loss += train(model=model, data=data, args=args)
    train_loss /= train_loader.__len__()

    eval_loss = 0
    for i in range(test_loader.__len__()):
        data = next(iter(test_loader))
        eval_loss += eval(model=model, data=data, args=args)
    eval_loss /= test_loader.__len__()
    
    print(f'Epoch {epoch}: train loss = {train_loss}; eval loss = {eval_loss}')
    train_loss = 0
    eval_loss = 0
    # else:
        # print(f'Epoch {epoch}.')
        
    if (epoch+1) % 20 == 0:
        torch.save(model.state_dict(), f'models/{model.name}_node2_epoch{start_epoch+epoch+1}.pth')